In [1]:
# badscript = '''contract get_fib_n(n:int){
#     let last_fib_calculated : int ;

#     entry get_fib(n: int){
#         let result: int = fib(n);
#         last_fib_calculated = result;
#         let a : int = b;
#     }

#     func fib(n: int) : int{
#         let x : int = last_fib_calculated;
#         if (n <= 1) {
#             return n;
#         }
#         else {
            
#             let a: int = n - 1;
#             let b: int = n - 2;
#             return fib(a) + fib(b);
#         }
#     }

#     func fib(n: int) : int{
#         let x : int = last_fib_calculated;
#     }
# }'''

In [2]:
script = '''
    contract test_address_type(){
        let temp: address;

        entry test_entry(a: int, b:nat, c:string, d:bool, e:address){
            temp = c;
            let f: address = e;
            let x: string = c + e;
            let y: address = "tz1icNwg34VPyWJevGRBz56r19MiqYeJQaxU";
            let z: address = "hola mundo";
            let z: address = y + z;
            let s: string = "hello";
            let v: address = test_func(s);
        }

        func test_func(a: address): address{
            return a;
        }
    }
    '''

In [3]:
from visitors.string_rep_visitor import FormatVisitor
from utils import run_tzscript_ast_building_pipeline

ast = run_tzscript_ast_building_pipeline(script)
format = FormatVisitor()
print(format.visit(ast))

0 	 <program> -> .contractid(<op-param-list{<stat_program_list>}, {'$'}
	 S' -> .<program>, {'$'} 

1 	 <program> -> contract.id(<op-param-list{<stat_program_list>}, {'$'} 

2 	 <program> -> contractid.(<op-param-list{<stat_program_list>}, {'$'} 

3 	 <param-list> -> .<param-list>,<param>, {')', ','}
	 <program> -> contractid(.<op-param-list{<stat_program_list>}, {'$'}
	 <param> -> .id:type, {')', ','}
	 <param-list> -> .<param>, {')', ','}
	 <op-param-list -> .), {'{'}
	 <op-param-list -> .<param-list>), {'{'} 

4 	 <op-param-list -> <param-list>.), {'{'}
	 <param-list> -> <param-list>.,<param>, {')', ','} 

5 	 <op-param-list -> <param-list>)., {'{'} 

6 	 <param> -> .id:type, {')', ','}
	 <param-list> -> <param-list>,.<param>, {')', ','} 

7 	 <param> -> id.:type, {')', ','} 

8 	 <param> -> id:.type, {')', ','} 

9 	 <param> -> id:type., {')', ','} 

10 	 <param-list> -> <param-list>,<param>., {')', ','} 

11 	 <program> -> contractid(<op-param-list.{<stat_program_list>}, {'$'} 

1

In [4]:
from visitors.semantic_check_visitor import SemanticCheckerVisitor
semantic_checker = SemanticCheckerVisitor()
semantic_checker.visit(ast)


string t
checkeo:  True
string h
checkeo:  False
string h
checkeo:  False


['Invalid type string for variable temp at line 6',
 ('Invalid types string and address for arithmetic operation at line 8',
  'PlusNode'),
 'Variable type address of z is not compatible with string at line 10',
 'Variable z is used, error in line 11',
 ('Invalid types address and address for arithmetic operation at line 11',
  'PlusNode'),
 'Invalid argument type string for function test_func at line 13, expected address']